## Data Extraction with Docling

In this notebook, we'll extract content from PDFs into structured formats:

- **Markdown**: Full document text with page breaks for chunking
- **Images**: Save pages containing large charts/diagrams (>500x500 pixels)
- **Tables**: Extract with 2 paragraphs of context + page number metadata

**Output Structure:**
```
data/rag-data/markdown/{company}/{document}.md
data/rag-data/images/{company}/{document}/page_5.png
data/rag-data/tables/{company}/{document}/table_1_page_5.md
```

https://github.com/docling-project/docling

### 1. Setup and Configuration

In [ ]:
from pathlib import Path
from typing import List, Tuple

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [ ]:
# Directory paths
DATA_DIR = "data/rag-data/pdfs"
OUTPUT_MD_DIR = "data/rag-data/markdown"
OUTPUT_IMAGES_DIR = "data/rag-data/images"
OUTPUT_TABLES_DIR = "data/rag-data/tables"

### Metadata Extraction

In [ ]:
def extract_metadata_from_filename(filename: str):
    """
    Extract metadata from filename.
    
    Expected format: CompanyName DocType [Quarter] Year.pdf
    Examples:
        - Amazon 10-Q Q1 2024.pdf
        - Microsoft 10-K 2023.pdf
    """

    filename = filename.replace('.pdf', '').replace('.md', '')
    parts = filename.split()

    return {
        'company_name': parts[0],
        'doc_type': parts[1],
        'fiscal_quarter': parts[2] if len(parts)==4 else None,
        'fiscal_year': parts[-1]
    }

extract_metadata_from_filename('apple 10-k 2023.pdf')

{'company_name': 'apple',
 'doc_type': '10-k',
 'fiscal_quarter': None,
 'fiscal_year': '2023'}

### Extract Markdown

In [ ]:
def convert_pdf_to_docling(pdf_file: Path):

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = 2
    pipeline_options.generate_picture_images = True
    pipeline_options.generate_page_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    return doc_converter.convert(pdf_file)

In [ ]:
def save_page_images(doc_converter, images_dir: Path):
    """
    Find and save pages with large images (>500x500 pixels).
    """

    pages_to_save = set()

    for item in doc_converter.document.iterate_items():
        element = item[0]

        if isinstance(element, PictureItem):
            image = element.get_image(doc_converter.document)

            if image.size[0]>500 and image.size[1]>500:
                page_no = element.prov[0].page_no if element.prov else None

                if page_no:
                    pages_to_save.add(page_no)


        # save images
        for page_no in pages_to_save:
            page = doc_converter.document.pages[page_no]

            page.image.pil_image.save(images_dir/ f"page_{page_no}.png", "PNG")


In [ ]:
def extract_context_and_table(lines: List[str], table_index: int):
    """
    Extract context and table content at a specific position.
    
    Args:
        lines: All markdown lines
        table_index: Where the table starts
    
    Returns:
        (combined_content, next_line_index)
    """

    table_lines = []
    i = table_index

    while (i < len(lines)) and (lines[i].startswith('|')):
        table_lines.append(lines[i])
        i = i + 1


    # previous 2 lines as table context
    start = max(0, table_index-2)
    context_lines = lines[start: table_index]

    content = '\n'.join(context_lines) + '\n\n' + '\n'.join(table_lines)

    return content, i
    

In [ ]:
def extract_tables_with_context(markdown_text: str):
    """
    Find all tables and extract them with context and page numbers.
    
    Returns:
        List of (content, table_name, page_number)
    """

    lines = markdown_text.split('\n')
    lines = [line for line in lines if line.strip()]
    tables = []
    current_page = 1
    table_num = 1
    i = 0

    while(i< len(lines)):
        # track page numbers
        if '<!-- page break -->' in lines[i]:
            current_page = current_page + 1
            i = i + 1
            continue

        # Table detected
        if lines[i].startswith('|') and lines[i].count('|')>1:
            content, next_i = extract_context_and_table(lines, i)

            tables.append((content, f"table_{table_num}", current_page))
            table_num = table_num + 1
            i = next_i

        else:
            i = i + 1


    return tables
    

In [ ]:
def save_tables(markdown_text, tables_dir):

    tables = extract_tables_with_context(markdown_text)

    for table_content, table_name, page_num in tables:
        content_with_page = f"**Page:** {page_num}\n\n{table_content}"
                
        (tables_dir/f"{table_name}_page_{page_num}.md").write_text(content_with_page, encoding='utf-8')


In [ ]:
def extract_pdf_content(pdf_file):
    metadata = extract_metadata_from_filename(pdf_file.stem)

    company_name = metadata['company_name']

    md_dir = Path(OUTPUT_MD_DIR) / company_name
    images_dir = Path(OUTPUT_IMAGES_DIR) / company_name / pdf_file.stem
    tables_dir = Path(OUTPUT_TABLES_DIR) / company_name / pdf_file.stem

    for dir_path in [md_dir, images_dir, tables_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)


    doc_converter = convert_pdf_to_docling(pdf_file)

    markdown_text = doc_converter.document.export_to_markdown(page_break_placeholder="<!-- page break -->")

    (md_dir / f"{pdf_file.stem}.md").write_text(markdown_text, encoding='utf-8')

    save_page_images(doc_converter, images_dir)

    save_tables(markdown_text, tables_dir)


In [ ]:
# pdf_file = Path('data\\rag-data\\pdfs\\apple\\apple 8-k q4 2023.pdf')

# extract_pdf_content(pdf_file)

data_path = Path(DATA_DIR)


In [ ]:
pdf_files = data_path.rglob("*.pdf")
for idx, pdf_file in enumerate(pdf_files):
    print(pdf_file)
    extract_pdf_content(pdf_file)

2025-12-15 16:24:48,684 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 16:24:48,687 - INFO - Going to convert document batch...
2025-12-15 16:24:48,688 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f076e025fec69d17629b0f4921f3a2a3
2025-12-15 16:24:48,688 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-15 16:24:48,698 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:24:48,702 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-15 16:24:48,702 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-15 16:24:48,745 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:24:48,747 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models

data\rag-data\pdfs\amazon\amazon 10-k 2023.pdf


2025-12-15 16:24:50,225 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:24:50,715 - INFO - Processing document amazon 10-k 2023.pdf
2025-12-15 16:25:32,804 - INFO - Finished converting document amazon 10-k 2023.pdf in 44.12 sec.
2025-12-15 16:25:32,998 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 16:25:33,001 - INFO - Going to convert document batch...
2025-12-15 16:25:33,001 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f076e025fec69d17629b0f4921f3a2a3
2025-12-15 16:25:33,002 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-15 16:25:33,015 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:25:33,019 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-15 16:25:33,019 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 

data\rag-data\pdfs\amazon\amazon 10-k 2024.pdf


2025-12-15 16:25:34,220 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:25:34,529 - INFO - Processing document amazon 10-k 2024.pdf
2025-12-15 16:26:02,461 - INFO - Finished converting document amazon 10-k 2024.pdf in 29.46 sec.
2025-12-15 16:26:02,656 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 16:26:02,659 - INFO - Going to convert document batch...
2025-12-15 16:26:02,660 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f076e025fec69d17629b0f4921f3a2a3
2025-12-15 16:26:02,661 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-15 16:26:02,673 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:26:02,677 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-15 16:26:02,678 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 

...

2025-12-15 16:39:49,933 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-15 16:39:49,934 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:39:50,872 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:39:51,118 - INFO - Processing document meta 10-q q3 2025.pdf
2025-12-15 16:39:59,302 - INFO - Finished converting document meta 10-q q3 2025.pdf in 9.63 sec.
2025-12-15 16:40:43,536 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 16:40:43,546 - INFO - Going to convert document batch...
2025-12-15 16:40:43,547 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f076e025fec69d17629b0f4921f3a2a3
2025-12-15 16:40:43,547 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-15 16:40:43,557 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:40:43,560 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12

data\rag-data\pdfs\meta\meta 10-q q4 2024.pdf


2025-12-15 16:40:43,745 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-15 16:40:43,745 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:40:44,747 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:40:45,025 - INFO - Processing document meta 10-q q4 2024.pdf
2025-12-15 16:40:52,733 - INFO - Finished converting document meta 10-q q4 2024.pdf in 9.20 sec.
2025-12-15 16:41:19,890 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 16:41:19,899 - INFO - Going to convert document batch...
2025-12-15 16:41:19,900 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f076e025fec69d17629b0f4921f3a2a3
2025-12-15 16:41:19,901 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-15 16:41:19,911 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-15 16:41:19,914 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12

data\rag-data\pdfs\meta\meta10-k 2023.pdf


2025-12-15 16:41:20,129 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-15 16:41:20,129 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:41:21,162 - INFO - Accelerator device: 'cuda:0'
2025-12-15 16:41:21,414 - INFO - Processing document meta10-k 2023.pdf
2025-12-15 16:42:28,167 - INFO - Finished converting document meta10-k 2023.pdf in 68.28 sec.
